In [1]:
import pandas as pd
import numpy as np
import re
import DataMiner
from DataMiner.Utils import ClearText
import pymongo

In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["ATFCredit"]
mycol = mydb["Txt_tables"]

In [4]:
table = mycol.find_one({"cnpj": str('83.109.579/0001-52')})

In [5]:
table.keys()

dict_keys(['_id', 'REGISTRO DE CONSULTAS', 'CINCO ULTIMAS CONSULTAS', 'HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)', 'PENDENCIAS', 'REFIN', 'PROTESTO', 'ERROR', 'cnpj', 'txt_file', 'result'])

In [6]:
pd.DataFrame(table['HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)'])

,MES/ANO,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,A_VISTA_QTD,A_VISTA_%,TOTAL
0,AGO/18,69463.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69463.0
1,JUL/18,249412.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249412.0
2,JUN/18,316116.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,316121.0
3,MAI/18,297089.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,297101.0
4,ABR/18,381179.0,100.0,0.0,0.0,440.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,381619.0
5,MAR/18,229399.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,229399.0
6,FEV/18,92083.0,100.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92095.0
7,JAN/18,78334.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228.0,78562.0
8,DEZ/17,82230.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82230.0
9,NOV/17,142147.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,142147.0


In [2]:
text = ""
with open("TxtPuro/01877691000179.txt") as infile:
    for line in infile:
        text += line

In [3]:
cpnj = text[text.find('CNPJ:'):].split('\n')[0]
cpnj = cpnj.replace('CNPJ:','').strip()
cpnj[:18]

'01.877.691/0001-79'

In [28]:
s = 'FEV/20'
# regexp = re.compile(r'[a-z,A-Z]')
# if regexp.search(s) and len(re.findall("[0-9]", s)) <= 3:

len(re.findall("[0-9]", s)) <= 2

True

In [32]:


def check_is_not_line(s):
    regexp = re.compile(r'[a-z,A-Z]')
    if regexp.search(s) and len(re.findall("[0-9]", s)) <= 3:
          return True
    return False

def check_is_not_data(s):
    if len(re.findall(r'[a-z,A-Z]', s)) <= 3  and len(re.findall("[0-9]", s)) <= 2:
          return False
    return True

def get_aux_values(aux, name, is_end = False):

#         print(aux)
        aux_i = np.where(aux == name)[0][0]

        aux_f = len(aux) - aux_i

        if not(is_end):
            aux_f = np.where(aux[aux_i:] == 'HISTORICO DE PAGAMENTOS')[0][0]

        aux_ = aux[aux_i:aux_i+aux_f] 
        j = 0
        if check_is_not_line(aux_[1]):
            j = 1
        try:
            return [aux_[1+j].split('|')[1].strip(), aux_[3+j] +' '+ aux_[4+j]]
    
        except:
            return [aux_[1+j].split('|')[1].strip(), np.nan]
    

def _capture_information(text):
    
    aux = re.split('  +',text)
    aux= np.array([i.replace('R451','') for i in aux])
#     print(aux)
    
    return {'MES/ANO':aux[4], 
            'PONTUAL_QTD':get_aux_values(aux, 'PONTUAL')[0],
            'PONTUAL_%': get_aux_values(aux, 'PONTUAL')[1],
            '8-15_QTD':get_aux_values(aux, '8-15')[0],
            '8-15_%':get_aux_values(aux, '8-15')[1],
            '16-30_QTD':get_aux_values(aux, '16-30')[0],
            '16-30_%':get_aux_values(aux, '16-30')[1],
            '31-60_QTD':get_aux_values(aux, '31-60')[0],
            '31-60_%':get_aux_values(aux, '31-60')[1],
            '+60_QTD':get_aux_values(aux, '+60')[0],
            '+60_%':get_aux_values(aux, '+60')[1],
            'A_VISTA_QTD':get_aux_values(aux, 'A VISTA')[0],
            'A_VISTA_%':get_aux_values(aux, 'A VISTA')[1],
            'TOTAL':get_aux_values(aux, 'TOTAL MES', is_end = True)[0]}


def create_df_by_text(text):
    aux_text = text
    index_base = aux_text.find('R451HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)')
    aux_text = aux_text[index_base:]

    aux_text = aux_text[aux_text.find('R452')+4:]
    aux_text = aux_text[aux_text.find('R452')+4:]

    lines = []

    while True:

        index_base = aux_text.find('R451')
        
        if index_base <= 0:
            break
            

        index_aux = aux_text[index_base+4:].find("R451TOTAL MES") + index_base
        index_aux = aux_text[index_aux:].find('HISTORICO DE PAGAMENTOS') + index_aux       
        
        aux_line = aux_text[index_base+4:index_aux]

        aux_text = aux_text[index_aux:].strip()

        if len(aux_line) <= 1:
            break
            
        
        try:
            new_line = _capture_information(aux_line)
            if check_is_not_data(new_line['MES/ANO']):
                break
            lines.append(new_line)
        except:
            continue


    df = pd.DataFrame(lines)
    df.loc[:,'PONTUAL_QTD':] = df.loc[:,'PONTUAL_QTD':].apply(ClearText.convert_text_to_float, axis = 0 )
    return df

In [33]:
text = ""
with open("TxtPuro/863529000139.txt") as infile:
    for line in infile:
        text += line

In [34]:
create_df_by_text(text)


,MES/ANO,PONTUAL_QTD,PONTUAL_%,8-15_QTD,8-15_%,16-30_QTD,16-30_%,31-60_QTD,31-60_%,+60_QTD,+60_%,A_VISTA_QTD,A_VISTA_%,TOTAL
0,FEV/20,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
1,JAN/20,300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000,NaN,300000
2,DEZ/19,200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,NaN,200000
3,NOV/19,100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100000,NaN,100000
4,OUT/19,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20000
5,SET/19,23000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23000
6,AGO/19,13000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13000
7,JUL/19,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000
8,JUN/19,13000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000,NaN,13000
9,MAI/19,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000


In [15]:


aux_text = text
index_base = aux_text.find('R451HISTORICO DE PAGAMENTOS NO MERCADO (VALORES EM R$)')
aux_text = aux_text[index_base:]

aux_text = aux_text[aux_text.find('R452')+4:]
aux_text = aux_text[aux_text.find('R452')+4:]



lines = []

In [16]:
aux_text

'ICB07 00  HISTORICO DE PAGAMENTOS                                                                              R451MES/ANO                                                                               0071                     R452ICF07 00  HISTORICO DE PAGAMENTOS                                                                              R451JAN/20                                                                                0072                     R452IDV07 00  HISTORICO DE PAGAMENTOS                                                                              R451PONTUAL       C12 |200 MIL A 300 MIL      |   95,00% A  97,00%      |                 0073                     R452IDV07 00  HISTORICO DE PAGAMENTOS                                                                              R4518-15              | -                     |      -         -        |                 0074                     R452IDV07 00  HISTORICO DE PAGAMENTOS                                              

In [9]:
index_base = aux_text.find('R451')

index_aux = aux_text[index_base+4:].find("R451TOTAL MES") + index_base
index_aux = aux_text[index_aux:].find('HISTORICO DE PAGAMENTOS') + index_aux       

aux_line = aux_text[index_base+4:index_aux]

aux_text = aux_text[index_aux:].strip()

aux_line

''

In [27]:
get_aux_values(aux, 'TOTAL MES', is_end = True)[0]

['' '0089' 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' 'NOV/19' '0090'
 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' 'PONTUAL'
 'C15 |500 MIL A 600 MIL' '|' '97,00% A 100,00%' '|' '0091' 'R452IDV07 00'
 'HISTORICO DE PAGAMENTOS' '8-15' '| -' '|' '-' '-' '|' '0092'
 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' '16-30' '| -' '|' '-' '-' '|'
 '0093' 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' '31-60' '| -' '|' '-' '-'
 '|' '0094' 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' '+60'
 'B16 |9 MIL A 9,5 MIL' '|' '1,00% A' '3,00%' '|' '0095' 'R452IDV07 00'
 'HISTORICO DE PAGAMENTOS' 'A VISTA' 'C11 |100 MIL A 200 MIL' '|' '-' '-'
 '|' '0096' 'R452IDV07 00' 'HISTORICO DE PAGAMENTOS' 'TOTAL MES'
 'C17 |700 MIL A 800 MIL' '|' '|' '0097' '']


'700 MIL A 800 MIL'

In [398]:

_capture_information(aux_line)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [333]:
aux_8_15

array(['8-15', 'B9', '|5,5 MIL A 6 MIL', '|', '1,00% A', '3,00%', '|',
       '0083', 'R452IDV07 00'], dtype='<U23')

In [316]:
import re
word = 'fubar'

def check_is_not_line(s):
    regexp = re.compile(r'[a-z,A-Z]')
    if regexp.search(s) and len(re.findall("[0-9]", s)) <= 3:
          return True
    return False
        
                


                


8

{'MES/ANO': 'SET/19',
 'PONTUAL_QTD': '200 MIL A 300 MIL',
 'PONTUAL_%': '93,00% A 95,00%',
 '8-15_QTD': '15 MIL A 17 MIL',
 '8-15_%': '5,00% A 7,00%',
 '16-30_QTD': ' -',
 '16-30_%': '- -',
 '31-60_QTD': ' -',
 '31-60_%': '- -',
 '+60_QTD': '250 A 300',
 '+60_%': '0,10% A 0,20%',
 'A_VISTA_QTD': '300 MIL A 400 MIL',
 'A_VISTA_%': '- -',
 'TOTAL': '500 MIL A 600 MIL'}

In [86]:
print(aux)

['', '0089', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', 'NOV/19', '0090', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', 'PONTUAL', 'C15 |500 MIL A 600 MIL', '|', '97,00% A 100,00%', '|', '0091', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', '8-15', '| -', '|', '-', '-', '|', '0092', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', '16-30', '| -', '|', '-', '-', '|', '0093', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', '31-60', '| -', '|', '-', '-', '|', '0094', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', '+60', 'B16 |9 MIL A 9,5 MIL', '|', '1,00% A', '3,00%', '|', '0095', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', 'A VISTA', 'C11 |100 MIL A 200 MIL', '|', '-', '-', '|', '0096', 'R452IDV07 00', 'HISTORICO DE PAGAMENTOS', 'TOTAL MES', 'C17 |700 MIL A 800 MIL', '|', '|', '0097', '']


In [59]:
index_aux

0

In [314]:
aux_text =  re.split('  +',aux_line)
aux_text

['20/09/2017 EXECUCAO',
 'R$',
 '3137.345',
 '00',
 '11',
 'SAO PAULO',
 'SP',
 '0679',
 '']

In [224]:
DATA
NATUREZA
AVAL
VALOR
DIST
VARA
CIDADE
UF
    
    aux_text = re.split('  +',text)
    
    
    j = 0
    aval = np.nan
    if len(aux_text) >= 10:
        j = 1
        aval = aux_text[1]

    return {'DATA':aux_text[0].split(' ')[0],\
                'NATUREZA':aux_text[0].split(' ')[1],\
                'AVAL':aval,\
               'VALOR':aux_text[2+j],\
               'DIST':aux_text[3+j],\
           'VARA':aux_text[4+j],\
           'CIDADE':aux_text[5+j],\
           'UF':aux_text[6+j],\}

'166.829 BDP0163230071190 BANRISUL                    0639                     T999000STRING PARCIAL - HA MAIS REGISTROS A ENVIAR                                                                \x03B49C      044699346000103JC     OO0000200            N99SCONAN                            SS   00000000000N        000000000000000          10000000000000000000000000100323195200000000000000000  202001271450542008009900001962B4CB49C030407C8A                                                000000000000000000000000000000003                                                                              R452ADV11 00 AIPZ0I038423414904469934                                                                              R45103/04/2018 EMPRESTIMO    N'

In [ ]:
{'DATA':text.split(' ')[0],\
            'MODALIDADE':text.split('  ')[0][11:], \
            'AVAL':text.split('  ')[1].split('R$')[0].strip(), \
            'VALOR': text.split('R$')[1].strip().split(' ')[0],
           'CONTRATO':text.split('R$')[1].strip().split(' ')[1],
           'ORIGEM':text.split('R$')[1].strip().split(' ')[2]}

In [ ]:
{'DATA':date,\
            'MODALIDADE':vencido, \
            'AVAL':vencer, \
            'VALOR': total,
           'CONTRATO': 
           'ORIGEM':}

In [84]:
aux_text

'ACF11 00  PENDENCIAS FINANCEIRAS                                                                               R45104/05/2018 OUTRAS OPER   N R$    232.821 BBH0210003934908 BANRISUL                    0638                     R452ADV11 00 AIPZ0I038995808004469934                                                                              R45123/04/2018 OUTRAS OPER   N R$    166.829 BDP0163230071190 BANRISUL                    0639                     T999000STRING PARCIAL - HA MAIS REGISTROS A ENVIAR                                                                \x03B49C      044699346000103JC     OO0000200            N99SCONAN                            SS   00000000000N        000000000000000          10000000000000000000000000100323195200000000000000000  202001271450542008009900001962B4CB49C030407C8A                                                000000000000000000000000000000003                                                                              R452ADV11 00 AIPZ0I038423

,DATA,MODALIDADE,AVAL,VALOR,CONTRATO,ORIGEM
0,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
1,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
2,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
3,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
4,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
5,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
6,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
7,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
8,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL
9,04/05/2018,OUTRAS OPER,N,232.821,BBH0210003934908,BANRISUL


In [91]:
aux_text = aux
date = aux_text[4:9]

index_base = aux_text.find("|")
index_aux = aux_text[index_base+1:].find("|")
vencido = aux_text[index_base:index_base+index_aux]
vencido = vencido.replace('|', '').strip()

aux_text = aux_text[index_base+index_aux+6:]
index_aux = aux_text.find("|")
vencer = aux_text[:index_aux].strip()



In [92]:
aux_text

'5 MI A 5,5 MI       |D10|8,5 MI A 9 MI              0565                 '

'8,5 MI A 9 M'

In [4]:
def capture_information(line):
    list_factoring_names = ['RNX', 'OPINIAO S/A', 'FIDC']
    line = line.replace('R302','')

    date = pd.to_datetime(line[:8], format = "%Y%m%d")
    name = line[8:-15]
    
    factoring_bool = False
    for factoring_name in list_factoring_names:
        if factoring_name in name:
            factoring_bool = True
            break
    
    return {'date':date,\
            'name':name, \
            'is_factoring':factoring_bool}

In [5]:
def create_df_by_lines(text):
    aux_text = text
    lines = []

    while True:
        index_base = aux_text.find('R252')

        if index_base <= 0:
            break

        index_aux = aux_text[index_base:].find('                          ')

        aux_line = aux_text[index_base:index_base+index_aux]
        aux_text = aux_text[index_base+index_aux:]
        
        lines.append(aux_line)
        
    return lines

In [6]:
aux_text = text
index_base = aux_text.find('R252')

In [8]:
index_base
aux = aux_text[index_base:]

a_vista = aux[21:41].strip()

In [9]:
df = create_df_by_lines(text)

In [10]:
df

['R252PONTUAL       A5 15 A 20             000000000000015000000000000020049000510020201219',
 'R2528-15          A1 1 A 3               000000000000001000000000000003001000030020201219',
 'R25216-30         A2 3 A 5               000000000000003000000000000005009000110020201219',
 'R25231-60         A3 5 A 10              000000000000005000000000000010021000230020201219',
 'R252+60           A3 5 A 10              000000000000005000000000000010013000150020201219',
 'R252A VISTA       A1 1 A 3               000000000000001000000000000003000000000020201219']